In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)  # Embedding layer
        lstm_out, _ = self.lstm(embedded)  # LSTM layer
        final_hidden_state = lstm_out[:, -1, :]  # Take the last hidden state
        output = self.sigmoid(self.fc(final_hidden_state))  # Fully connected + sigmoid
        return output

# Custom Dataset class
class ClaimsDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

# Training function
def train_model(model, dataloader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device).float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}")

# Evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predictions.extend(outputs.squeeze().cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    mae = np.mean(np.abs(np.array(predictions) - np.array(true_labels)))
    print(f"MAE for Propensity Scores: {mae:.4f}")
    return mae

# Mock Data for testing
num_samples = 1000
sequence_length = 10
vocab_size = 50
sequences = np.random.randint(0, vocab_size, (num_samples, sequence_length))
labels = np.random.randint(0, 2, num_samples)

# Create Dataset and DataLoader
dataset = ClaimsDataset(torch.tensor(sequences, dtype=torch.long), torch.tensor(labels, dtype=torch.float32))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Model configuration
input_dim = vocab_size
embedding_dim = 64
hidden_dim = 64
num_layers = 2
num_epochs = 10

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_dim, embedding_dim, hidden_dim, num_layers).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train and evaluate
train_model(model, dataloader, criterion, optimizer, num_epochs)
evaluate_model(model, dataloader)

Epoch 1/10, Loss: 0.6930
Epoch 2/10, Loss: 0.6878
Epoch 3/10, Loss: 0.6785
Epoch 4/10, Loss: 0.6608
Epoch 5/10, Loss: 0.6383
Epoch 6/10, Loss: 0.6094
Epoch 7/10, Loss: 0.5530
Epoch 8/10, Loss: 0.5059
Epoch 9/10, Loss: 0.4224
Epoch 10/10, Loss: 0.3348
MAE for Propensity Scores: 0.1682


0.16823071

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [3]:
data = pd.read_csv("C:/Users/rafif/Box/CS6350 project/data/linear_regression.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/rafif/Box/CS6350 project/data/linear_regression.csv'

In [ ]:
# Step 2: Determine Top 10 Added Medications
medication_column = "added_combined_meds" 
top_10_meds = data[medication_column].value_counts().head(10).index.tolist()

# Filter data for only top 10 medications
data = data[data[medication_column].isin(top_10_meds)]

In [ ]:

# Step 3: Feature Preparation
features = [
    "race_ethnicity", "sex", "marital_status", "patient_regional_location",
    "hba1c_last_value", "diabetes_duration_2023", "Last_Med_ME", "Last_Med_SUL",
    "Last_Med_SGLT2", "Last_Med_Insulin", "Last_Med_AlphaGI", "Last_Med_DPP4",
    "Last_Med_GLP1", "Last_Med_GIP_GLP1", "Last_Med_TZD"
]
outcome = "Y"  # Replace with the actual column for the outcome

In [ ]:
# Encode categorical variables and scale numeric ones
categorical_features = ["race_ethnicity", "sex", "marital_status", "patient_regional_location"]
numeric_features = [f for f in features if f not in categorical_features]

# Encoding categorical features
for col in categorical_features:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

In [ ]:
print(data.columns)

In [ ]:
numeric_features = [
    "hba1c_last_value", "diabetes_duration_2023", "Last_Med_MET", "Last_Med_SUL",
    "Last_Med_SGLT2", "Last_Med_Insulin", "Last_Med_AlphaGI", "Last_Med_DPP4",
    "Last_Med_GLP1", "Last_Med_GIP_GLP1", "Last_Med_TZD"
]

In [ ]:
scaler = StandardScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])

In [ ]:
features = [
    "race_ethnicity", "sex", "marital_status", "patient_regional_location",
    "hba1c_last_value", "diabetes_duration_2023", "Last_Med_MET", "Last_Med_SUL",
    "Last_Med_SGLT2", "Last_Med_Insulin", "Last_Med_AlphaGI", "Last_Med_DPP4",
    "Last_Med_GLP1", "Last_Med_GIP_GLP1", "Last_Med_TZD"
]

In [ ]:
# Step 4: Multinomial Logistic Regression for Propensity Score
X = data[features]  # Features for modeling
y = data[medication_column]  # Target (medication, multiclass)

In [ ]:
missing_features = [col for col in features if col not in data.columns]
print(f"Missing features: {missing_features}")

In [ ]:
features = [col for col in features if col in data.columns]

In [ ]:
medication_column = "added_combined_meds"

In [ ]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Multinomial Logistic Regression
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model.fit(X_train, y_train)

# Step 5: Predict IPTW Vector
propensity_scores = model.predict_proba(X_test)  # Propensity score for each medication

# Map y_test to integer indices
class_mapping = {label: idx for idx, label in enumerate(model.classes_)}
mapped_y_test = y_test.map(class_mapping)

# Compute IPTW using integer indices
iptw = 1 / propensity_scores[np.arange(len(mapped_y_test)), mapped_y_test]

# Normalize IPTW for stability (optional, for extreme values)
iptw = np.clip(iptw, a_min=0.01, a_max=100)

# Step 6: Calculate Outcome and Validate Model
log_loss_value = log_loss(y_test, propensity_scores)
print(f"Log Loss for Multinomial Logistic Regression: {log_loss_value:.4f}")

# Add IPTW and propensity scores to test data
result_df = X_test.copy()
result_df['True_Medication'] = y_test
result_df['IPTW'] = iptw
for idx, class_label in enumerate(model.classes_):
    result_df[f"Propensity_Score_{class_label}"] = propensity_scores[:, idx]

print(result_df.head())
print("Unique values in y_test:", y_test.unique())
print("Model classes:", model.classes_)


In [ ]:
# Save result_df to a CSV file
output_filename = "multinomial_logistic_results.csv"
result_df.to_csv(output_filename, index=False)
print(f"Results saved to {output_filename}")

# Provide download link (for Jupyter Notebook environment)
import shutil
shutil.move(output_filename, f"/mnt/data/{output_filename}")
print(f"File is available for download: {output_filename}")

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

# Load datasets
train_data = pd.read_csv(r"C:\Users\rafif\Box\CS6350 project\data\train.csv")
test_data = pd.read_csv(r"C:\Users\rafif\Box\CS6350 project\data\test.csv")

# Predictive Features based on given parameters
predictive_features = [
    "hba1c_last_value",  # Last HbA1C value
    "race_aian", "race_black", "race_hispanic", "race_nhpi", "race_white",  # Demographic variables
    "sex_M",  # Sex
    "marital_married", "marital_unknown", "marital_single",  # Marital status
    "region_midwest", "region_south", "region_west",  # Geographical info
    "diabetes_duration_2023",  # Diabetes duration
    "Last_Med_MET", "Last_Med_SUL", "Last_Med_SGLT2", "Last_Med_Insulin", 
    "Last_Med_AlphaGI", "Last_Med_DPP4", "Last_Med_GLP1", "Last_Med_GIP_GLP1", "Last_Med_TZD"  # Medications
]

# Target columns (Medication Added)
added_med_columns = [
    "Added_Med_MET", "Added_Med_SUL", "Added_Med_SGLT2", "Added_Med_Insulin",
    "Added_Med_DPP4", "Added_Med_GLP1", "Added_Med_GIP_GLP1", "Added_Med_TZD", "Added_Nothing"
]

# Handle imbalance in dataset (Resampling)
majority_class = train_data[train_data['Added_Nothing'] == 1]
minority_classes = train_data[train_data['Added_Nothing'] == 0]

# Downsample majority class
majority_downsampled = resample(
    majority_class,
    replace=False,
    n_samples=500,  # Adjust to desired sample size
    random_state=42
)

# Upsample minority classes
minority_oversampled = resample(
    minority_classes,
    replace=True,
    n_samples=500,  # Adjust to match majority class size
    random_state=42
)

# Combine and shuffle the balanced dataset
balanced_train_data = pd.concat([majority_downsampled, minority_oversampled])
balanced_train_data = balanced_train_data.sample(frac=1, random_state=42)

# Prepare training features and labels
X_train = balanced_train_data[predictive_features].values
y_train = balanced_train_data[added_med_columns].values

# Prepare test features and labels
X_test = test_data[predictive_features].values
y_test = test_data[added_med_columns].values

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(added_med_columns), activation='softmax')  # One output neuron per medication
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

# Evaluate the model
train_loss, train_accuracy = model.evaluate(X_train, y_train)
print(f"Train Loss: {train_loss:.2f}")
print(f"Train Accuracy: {train_accuracy:.2f}")
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")

# Visualization - Plot training, validation, and testing loss in one graph
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='green')

# Use the test set to calculate loss for each epoch
test_losses = [model.evaluate(X_test, y_test, verbose=0)[0]] * 30  # Same test loss for all epochs

plt.plot(test_losses, label='Test Loss', color='red')

# Highlight the best model (validation loss minimum) with a curve-like representation
best_epoch = np.argmin(history.history['val_loss'])  # Find the epoch with the minimum validation loss
best_val_loss = min(history.history['val_loss'])

# Plot best validation loss as part of the curve
best_val_loss_curve = [best_val_loss] * 30  # Create a constant array for best validation loss
plt.plot(best_val_loss_curve, label=f"Best Validation Loss (Epoch {best_epoch+1})", color='purple', linestyle='--')

# Labels and title
plt.title('Neural Network Cost Function Dynamics (Training, Validation, Testing)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Epoch 1/30


C:\Users\rafif\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.2192 - loss: 2.2665 - val_accuracy: 0.5050 - val_loss: 1.8631
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5058 - loss: 1.8496 - val_accuracy: 0.5050 - val_loss: 1.7417
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5035 - loss: 1.7394 - val_accuracy: 0.5200 - val_loss: 1.6659
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4704 - loss: 1.7451 - val_accuracy: 0.5300 - val_loss: 1.6035
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5237 - loss: 1.6241 - val_accuracy: 0.5350 - val_loss: 1.5527
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5573 - loss: 1.5303 - val_accuracy: 0.5900 - val_loss: 1.5259
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5699 - loss: 1.4936 - val_accuracy: 0.6050 - val_loss: 1.4983
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5901 - loss: 1.4020 - val_accuracy: 0.5900 - val_loss: 1.4736
Epo